<a href="https://colab.research.google.com/github/RifqiHafizuddin/Computational-Intelligence-week3/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Rifqi Hafizuddin

NPM: 2106638204

output: https://drive.google.com/file/d/17S44UIF-gQa0Yw7yxb7vQfLvNqUv7sq9/view?usp=sharing

In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python-headless
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 16.48 MiB/s, done.
Resolving deltas: 100% (11695/11695), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.5/877.5 kB 45.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Use the video path from your drive
video_path = '/content/drive/MyDrive/video_utski.mp4'
# Video Link: https://www.youtube.com/watch?v=b6CWJ0gqoMM

Mounted at /content/drive


In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True).to(device)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-10-25 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 262MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
import cv2
import torch
import pandas as pd

# Load the video
video_capture = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not video_capture.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

# Initialize VideoWriter to save the output video with bounding boxes
output_video = cv2.VideoWriter('output_with_detections.mp4',
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               fps,
                               (frame_width, frame_height))

# Initialize an empty list to store detection results
detection_results = []

frame_index = 0  # Track frame number

while True:
    ret, frame = video_capture.read()

    if not ret:  # Exit if no frame is returned
        print("End of video or error reading frame.")
        break

    # Check if frame is valid
    if frame is None:
        print("Warning: Received an empty frame.")
        continue

    # Convert frame to RGB (YOLO expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # YOLOv5 inference on the frame
    results = model(frame_rgb)

    # Extract bounding boxes, confidence scores, and class labels
    bbox_data = results.xyxy[0].cpu().numpy()  # Get boxes, confidences, and labels

    # Process each detection in the current frame
    for bbox in bbox_data:
        x1, y1, x2, y2, confidence, cls = bbox
        label = model.names[int(cls)]  # Get class name

         # Draw bounding box and label on the frame
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} {confidence:.2f}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Normalize bounding box coordinates
        height, width, _ = frame.shape
        x1_norm = x1 / width
        y1_norm = y1 / height
        x2_norm = x2 / width
        y2_norm = y2 / height

        # Append the detection result as a dictionary
        detection_results.append({
            'frame': frame_index,
            'x1': x1_norm,
            'y1': y1_norm,
            'x2': x2_norm,
            'y2': y2_norm,
            'confidence': confidence,
            'class': label
        })

    frame_index += 1
    output_video.write(frame)

# Convert the list of detection results to a pandas DataFrame
df_detections = pd.DataFrame(detection_results)

# Display the DataFrame
print(df_detections)

# Optionally, save the DataFrame to a CSV file
df_detections.to_csv('object_detection_results.csv', index=False)

# Release resources
video_capture.release()
output_video.release()

print("Object detection data saved to DataFrame.")

Streaming output truncated to the last 5000 lines.
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/

End of video or error reading frame.
       frame        x1        y1        x2        y2  confidence  class
0        120  0.547593  0.621374  0.584616  0.677268    0.805414    car
1        120  0.482904  0.625555  0.506091  0.652820    0.689071    car
2        121  0.548017  0.621633  0.584944  0.677157    0.833068    car
3        121  0.484044  0.622190  0.505634  0.650816    0.722777    car
4        122  0.547275  0.620369  0.585641  0.676084    0.826810    car
...      ...       ...       ...       ...       ...         ...    ...
15967   3716  0.000000  0.594429  0.027055  0.671170    0.670317    car
15968   3716  0.587134  0.672596  0.687187  0.784959    0.618334  truck
15969   3716  0.516345  0.668620  0.538722  0.699561    0.540238    car
15970   3716  0.421877  0.663875  0.437376  0.686711    0.372418    car
15971   3716  0.590752  0.672366  0.688111  0.783941    0.284112    car

[15972 rows x 7 columns]
Object detection data saved to DataFrame.


In [ ]:
df_detections

,frame,x1,y1,x2,y2,confidence,class
0,120,0.547593,0.621374,0.584616,0.677268,0.805414,car
1,120,0.482904,0.625555,0.506091,0.652820,0.689071,car
2,121,0.548017,0.621633,0.584944,0.677157,0.833068,car
3,121,0.484044,0.622190,0.505634,0.650816,0.722777,car
4,122,0.547275,0.620369,0.585641,0.676084,0.826810,car
...,...,...,...,...,...,...,...
15967,3716,0.000000,0.594429,0.027055,0.671170,0.670317,car
15968,3716,0.587134,0.672596,0.687187,0.784959,0.618334,truck
15969,3716,0.516345,0.668620,0.538722,0.699561,0.540238,car
15970,3716,0.421877,0.663875,0.437376,0.686711,0.372418,car


from matplotlib import pyplot as plt
df_detections['frame'].plot(kind='hist', bins=20, title='frame')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_detections['x1'].plot(kind='hist', bins=20, title='x1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_detections['y1'].plot(kind='hist', bins=20, title='y1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_detections['x2'].plot(kind='hist', bins=20, title='x2')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['frame']
  ys = series['y2']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_detections.sort_values('frame', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('frame')
_ = plt.ylabel('y2')

from matplotlib import pyplot as plt
df_detections['frame'].plot(kind='line', figsize=(8, 4), title='frame')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_detections['x1'].plot(kind='line', figsize=(8, 4), title='x1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_detections['y1'].plot(kind='line', figsize=(8, 4), title='y1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_detections['x2'].plot(kind='line', figsize=(8, 4), title='x2')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
df_detections.groupby(['class']).mean()[['confidence']]

,confidence
class,
airplane,0.390079
bicycle,0.320543
bird,0.333785
boat,0.261877
bus,0.516474
car,0.565099
clock,0.380940
cow,0.357241
fire hydrant,0.328048


In [ ]:
df_detections.groupby(['class']).mean()

,frame,x1,y1,x2,y2,confidence
class,,,,,,
airplane,2542.000000,0.043381,0.000461,0.103416,0.078264,0.390079
bicycle,1896.333333,0.623674,0.670780,0.651848,0.733507,0.320543
bird,2635.600000,0.373854,0.373560,0.412000,0.419566,0.333785
boat,3602.000000,0.087632,0.562860,0.202697,0.670992,0.261877
bus,2659.500000,0.557909,0.566328,0.726004,0.791014,0.516474
car,2200.304490,0.432019,0.657131,0.483240,0.720902,0.565099
clock,2373.333333,0.405882,0.160624,0.432527,0.244544,0.380940
cow,975.833333,0.811284,0.615748,0.852537,0.674991,0.357241
fire hydrant,2356.812500,0.377070,0.700063,0.397943,0.764655,0.328048


In [ ]:
from google.colab import files

# Download the video to your local machine
files.download('output_with_detections.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('object_detection_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>